In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glob import glob 

import pandas as pd

from fastcore.xtras import load_pickle

import matplotlib.pyplot as plt 
plt.style.use(['science', 'nature'])

## Load results

In [3]:
all_out = glob("out/**/*.pkl")

In [4]:
all_out = [load_pickle(p) for p in all_out]

In [7]:
extracted_res = []
xgboost_results = []
tabpfn_results = []

for out in all_out: 
    res = {
        'train_size': out['train_size'],
        'frac_valid': out['frac_valid'],
        'accuracy': out['accuracy'],
        'f1_macro': out['f1_macro'],
        'f1_micro': out['f1_micro'],
        'kappa': out['kappa'],
        "num_classes": out['num_classes']
    }

    xgb_res = {
        'train_size': out['train_size'], 
        'accuracy': out['xgboost_baseline']['accuracy'], 
        'f1_macro': out['xgboost_baseline']['f1_macro'],
        'f1_micro': out['xgboost_baseline']['f1_micro'],
        'kappa': out['xgboost_baseline']['kappa'],
        "num_classes": out['num_classes']
    }

    tabpfn_res = {
        'train_size': out['train_size'], 
        'accuracy': out['tabpfn_baseline']['accuracy'], 
        'f1_macro': out['tabpfn_baseline']['f1_macro'],
        'f1_micro': out['tabpfn_baseline']['f1_micro'],
        'kappa': out['tabpfn_baseline']['kappa'],
        "num_classes": out['num_classes']
    }

    extracted_res.append(res)
    xgboost_results.append(xgb_res)
    tabpfn_results.append(tabpfn_res)

In [8]:
res = pd.DataFrame(extracted_res)
xgboost_res = pd.DataFrame(xgboost_results)
tabpfn_res = pd.DataFrame(tabpfn_results)

In [9]:
res

,train_size,frac_valid,accuracy,f1_macro,f1_micro,kappa
0,10,1.0,0.560,0.544581,0.560,0.120
1,50,1.0,0.784,0.783876,0.784,0.568
2,100,1.0,0.940,0.939976,0.940,0.880


In [11]:
xgboost_res

,train_size,accuracy,f1_macro,f1_micro,kappa
0,10,0.500,0.333333,0.500,0.000
1,50,0.812,0.811852,0.812,0.624
2,100,0.876,0.875982,0.876,0.752


In [12]:
tabpfn_res

,train_size,accuracy,f1_macro,f1_micro,kappa
0,10,0.844,0.841234,0.844,0.688
1,50,0.884,0.883983,0.884,0.768
2,100,0.936,0.935984,0.936,0.872
